In [1]:
import logging

from absl import app
from absl import flags
import apache_beam as beam
import xarray as xr
import xarray_beam as xbeam
import xee

import ee
import geemap

import pyproj

import os
import xarray as xr
import geopandas as gp
import pandas as pd
import numpy as np
import sparse

import hvplot.pandas
import hvplot.xarray
import dask
import cf_xarray

from pynhd import NLDI, WaterData
import intake
import cartopy.crs as ccrs
from shapely.geometry import Polygon

import time
import dask
import requests
from dask.distributed import Client

import requests
from IPython.display import HTML
import warnings
from IPython.display import HTML

from pynhd import NLDI, WaterData
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx

In [2]:
geemap.ee_initialize()
ee.Initialize()

In [3]:
# open the hytest data intake catalog
hytest_cat = intake.open_catalog("https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml")
list(hytest_cat)


# open the conus404 sub-catalog
# cat = hytest_cat['conus404-catalog']
cat = hytest_cat['conus404-catalog']
list(cat)

## Select the dataset you want to read into your notebook and preview its metadata
#  conus404-daily-osn
dataset = 'conus404-daily-osn' 
cat[dataset]

print(f"Reading {dataset} metadata...", end='')
ds = cat[dataset].to_dask().metpy.parse_cf()
print("done")

# ds_conus = ds.ACRAINLSM.sel(time=slice('2010-01-01','2020-12-31')).load()
ds_conus = ds.sel(time=slice('2000-01-01','2010-12-31'))
ds_conus

Reading conus404-daily-osn metadata...

C:\Users\adi10136\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


done


<xarray.Dataset>
Dimensions:         (y: 1015, x: 1367, time: 4018, bottom_top_stag: 51,
                     bottom_top: 50, soil_layers_stag: 4, x_stag: 1368,
                     y_stag: 1016, snow_layers_stag: 3, snso_layers_stag: 7)
Coordinates:
    lat             (y, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
    lon             (y, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
  * time            (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2010-12-31
  * x               (x) float64 -2.732e+06 -2.728e+06 ... 2.728e+06 2.732e+06
  * y               (y) float64 -2.028e+06 -2.024e+06 ... 2.024e+06 2.028e+06
    metpy_crs       object Projection: lambert_conformal_conic
    lat_u           (y, x_stag) float32 dask.array<chunksize=(350, 175), meta=np.ndarray>
    lon_u           (y, x_stag) float32 dask.array<chunksize=(350, 175), meta=np.ndarray>
    lat_v           (y_stag, x) float32 dask.array<chunksize=(175, 350), meta=np.ndarray>
    lon_v           (y_stag, x) float32 dask.array<chunksize=(175, 350), meta=np.ndarray>
Dimensions without coordinates: bottom_top_stag, bottom_top, soil_layers_stag,
                                x_stag, y_stag, snow_layers_stag,
                                snso_layers_stag
Data variables: (12/153)
    ACDEWC          (time, y, x) float32 dask.array<chunksize=(19, 350, 350), meta=np.ndarray>
    ACDRIPR         (time, y, x) float32 dask.array<chunksize=(19, 350, 350), meta=np.ndarray>
    ACDRIPS         (time, y, x) float32 dask.array<chunksize=(19, 350, 350), meta=np.ndarray>
    ACECAN          (time, y, x) float32 dask.array<chunksize=(19, 350, 350), meta=np.ndarray>
    ACEDIR          (time, y, x) float32 dask.array<chunksize=(19, 350, 350), meta=np.ndarray>
    ACETLSM         (time, y, x) float32 dask.array<chunksize=(19, 350, 350), meta=np.ndarray>
    ...              ...
    ZNU             (bottom_top) float32 dask.array<chunksize=(50,), meta=np.ndarray>
    ZNW             (bottom_top_stag) float32 dask.array<chunksize=(51,), meta=np.ndarray>
    ZS              (soil_layers_stag) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    ZSNSO           (time, snso_layers_stag, y, x) float32 dask.array<chunksize=(19, 7, 350, 350), meta=np.ndarray>
    ZWT             (time, y, x) float32 dask.array<chunksize=(19, 350, 350), meta=np.ndarray>
    crs             int64 ...
Attributes: (12/148)
    AER_ANGEXP_OPT:                  1
    AER_ANGEXP_VAL:                  1.2999999523162842
    AER_AOD550_OPT:                  1
    AER_AOD550_VAL:                  0.11999999731779099
    AER_ASY_OPT:                     1
    AER_ASY_VAL:                     0.8999999761581421
    ...                              ...
    WEST-EAST_PATCH_START_STAG:      1
    WEST-EAST_PATCH_START_UNSTAG:    1
    W_DAMPING:                       1
    YSU_TOPDOWN_PBLMIX:              0
    history:                         Tue Mar 29 16:35:22 2022: ncrcat -A -vW ...
    history_of_appended_files:       Tue Mar 29 16:35:22 2022: Appended file ...

In [4]:
dataset = ee.ImageCollection("OREGONSTATE/PRISM/AN81d").filterDate("2000-01-01", "2010-12-31")
ds = geemap.ee_to_xarray(dataset, crs="EPSG:4326", scale=0.25)

ds = ds.chunk({'time': 10})
#  print(f"Reading {dataset} metadata...", end='')
ds = ds.metpy.parse_cf()
print("done")

ds_prism = ds
ds_prism

done


<xarray.Dataset>
Dimensions:    (time: 4017, lon: 1440, lat: 720)
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01T12:00:00 ... 2010-12-30T12:00:00
  * lon        (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * lat        (lat) float64 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
    metpy_crs  object Projection: latitude_longitude
Data variables:
    ppt        (time, lon, lat) float32 dask.array<chunksize=(10, 1440, 720), meta=np.ndarray>
    tmean      (time, lon, lat) float32 dask.array<chunksize=(10, 1440, 720), meta=np.ndarray>
    tmin       (time, lon, lat) float32 dask.array<chunksize=(10, 1440, 720), meta=np.ndarray>
    tmax       (time, lon, lat) float32 dask.array<chunksize=(10, 1440, 720), meta=np.ndarray>
    tdmean     (time, lon, lat) float32 dask.array<chunksize=(10, 1440, 720), meta=np.ndarray>
    vpdmin     (time, lon, lat) float32 dask.array<chunksize=(10, 1440, 720), meta=np.ndarray>
    vpdmax     (time, lon, lat) float32 dask.array<chunksize=(10, 1440, 720), meta=np.ndarray>
Attributes: (12/20)
    date_range:               [347155200000, 1647648000000]
    description:              <p>The PRISM daily and monthly datasets are gri...
    keywords:                 ['climate', 'daily', 'geophysical', 'oregonstat...
    period:                   1
    period_mapping:           [347155200000, 1647648000000]
    product_tags:             ['precipitation', 'climate', 'weather', 'temper...
    ...                       ...
    visualization_0_bands:    ppt
    visualization_0_max:      50.0
    visualization_0_min:      0.0
    visualization_0_name:     Precipitation
    visualization_0_palette:  red,yellow,green,cyan,purple
    crs:                      EPSG:4326

In [5]:
ds_prism = ds_prism.drop_vars('metpy_crs')

In [6]:
## lLC spatial reference system
x = 'x'
y = 'y'



In [10]:
crs_info = ds_conus.crs
xx = ds_conus.x.values
yy = ds_conus.y.values
globe = ccrs.Globe(ellipse='sphere', semimajor_axis=6370000, semiminor_axis=6370000)
lcc = ccrs.LambertConformal(globe=globe,
                            central_longitude=crs_info.longitude_of_central_meridian, 
                            central_latitude=crs_info.latitude_of_projection_origin,
                            standard_parallels=crs_info.standard_parallel)
print(xx)
print(yy)

[-2732000. -2728000. -2724000. ...  2724000.  2728000.  2732000.]
[-2028000. -2024000. -2020000. ...  2020000.  2024000.  2028000.]


In [11]:
crs_info.standard_parallel

[30.0, 50.0]

In [12]:
crs_info.latitude_of_projection_origin

39.100006103515625

In [13]:
crs_info.longitude_of_central_meridian

-97.9000015258789

In [14]:
lcc_wkt = lcc.to_wkt()

In [15]:

# Reproject PRISM dataset to Lambert Conformal
source_proj = pyproj.Proj('EPSG:4326')
target_proj = pyproj.Proj(lcc.proj4_init)

# Function to reproject coordinates
def reproject_coordinates(lon, lat, src_proj, tgt_proj):
    transformer = pyproj.Transformer.from_proj(src_proj, tgt_proj, always_xy=True)
    return transformer.transform(lon, lat)

# Extract lat/lon values
lon = ds_prism['lon'].values
lat = ds_prism['lat'].values

# Create meshgrid for lat/lon
lon_mesh, lat_mesh = np.meshgrid(lon, lat)

# Reproject coordinates
lon_reproj, lat_reproj = reproject_coordinates(lon_mesh, lat_mesh, source_proj, target_proj)

# Step 5: Prepare the reprojected PRISM dataset
ds_prism_reproj = ds_prism.assign_coords({'lon_reproj': (('lat', 'lon'), lon_reproj), 'lat_reproj': (('lat', 'lon'), lat_reproj)})

# Interpolate PRISM data to CONUS404 grid
interp_data = {}
for var in ds_prism.data_vars:
    interp_data[var] = ds_prism[var].interp(lon=ds_conus['lon'], lat=ds_conus['lat'], method='nearest')

ds_prism_interp = xr.Dataset(interp_data, coords={'lon': ds_conus['lon'], 'lat': ds_conus['lat'], 'time': ds_prism['time']})


C:\Users\adi10136\AppData\Local\anaconda3\Lib\site-packages\xarray\core\missing.py:715: PerformanceWarning: Increasing number of chunks by factor of 12
  _, rechunked = chunkmanager.unify_chunks(*args)
C:\Users\adi10136\AppData\Local\anaconda3\Lib\site-packages\xarray\core\missing.py:715: PerformanceWarning: Increasing number of chunks by factor of 12
  _, rechunked = chunkmanager.unify_chunks(*args)
C:\Users\adi10136\AppData\Local\anaconda3\Lib\site-packages\xarray\core\missing.py:715: PerformanceWarning: Increasing number of chunks by factor of 12
  _, rechunked = chunkmanager.unify_chunks(*args)
C:\Users\adi10136\AppData\Local\anaconda3\Lib\site-packages\xarray\core\missing.py:715: PerformanceWarning: Increasing number of chunks by factor of 12
  _, rechunked = chunkmanager.unify_chunks(*args)
C:\Users\adi10136\AppData\Local\anaconda3\Lib\site-packages\xarray\core\missing.py:715: PerformanceWarning: Increasing number of chunks by factor of 12
  _, rechunked = chunkmanager.unify_chun

In [16]:
ds_conus['time'] = ds_conus.indexes['time'].normalize()
ds_prism_interp['time'] = ds_prism_interp.indexes['time'].normalize()

In [17]:
var_prism = 'tmean'  
var_conus = 'T2'

# Debug: Print time dimensions before alignment
print("CONUS404 time dimension:", ds_conus['time'].values)
print("PRISM time dimension:", ds_prism_interp['time'].values)

common_times = np.intersect1d(ds_conus['time'].values, ds_prism_interp['time'].values)
print("Common time periods:", common_times)


CONUS404 time dimension: ['2000-01-01T00:00:00.000000000' '2000-01-02T00:00:00.000000000'
 '2000-01-03T00:00:00.000000000' ... '2010-12-29T00:00:00.000000000'
 '2010-12-30T00:00:00.000000000' '2010-12-31T00:00:00.000000000']
PRISM time dimension: ['2000-01-01T00:00:00.000000000' '2000-01-02T00:00:00.000000000'
 '2000-01-03T00:00:00.000000000' ... '2010-12-28T00:00:00.000000000'
 '2010-12-29T00:00:00.000000000' '2010-12-30T00:00:00.000000000']
Common time periods: ['2000-01-01T00:00:00.000000000' '2000-01-02T00:00:00.000000000'
 '2000-01-03T00:00:00.000000000' ... '2010-12-28T00:00:00.000000000'
 '2010-12-29T00:00:00.000000000' '2010-12-30T00:00:00.000000000']


In [18]:
common_times = np.intersect1d(ds_conus['time'].values, ds_prism_interp['time'].values)
ds_conus = ds_conus.sel(time=common_times)
ds_prism_interp = ds_prism_interp.sel(time=common_times)

In [19]:
ds_conus = ds_conus.sel(time=common_times)

In [20]:
aligned_conus, aligned_prism = xr.align(ds_conus[var_conus], ds_prism_interp[var_prism], join='inner')

In [21]:
Diff = (aligned_conus - 273.15) - (aligned_prism)
print(Diff)

<xarray.DataArray (time: 4017, y: 1015, x: 1367)>
dask.array<sub, shape=(4017, 1015, 1367), dtype=float32, chunksize=(10, 350, 350), chunktype=numpy.ndarray>
Coordinates:
    lat        (y, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
  * time       (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2010-12-30
  * x          (x) float64 -2.732e+06 -2.728e+06 ... 2.728e+06 2.732e+06
  * y          (y) float64 -2.028e+06 -2.024e+06 ... 2.024e+06 2.028e+06
    metpy_crs  object Projection: lambert_conformal_conic


In [22]:
%%time
ds = Diff
ds_mean = ds.mean(dim='time')
# ds_mean.hvplot(x='lon', y='y', rasterize=True, cmap='viridis', frame_width=600, tiles='OpenTopoMap', title='Annual average precipitation in mm (2010-2020) CONUS404 - PRISM', alpha=0.5)
# ds_mean.hvplot.quadmesh(x='lon', y='lat', rasterize=True, geo=True, tiles='OSM', title = 'Monthly Average Daily Minimum 2-m Temperature CONUS404-PRISM (C)', cmap='viridis').opts('Image', alpha=0.6)
ds_mean.hvplot.quadmesh(x='lon', y='lat', rasterize=True, geo=True, tiles='OSM', title = 'Monthly Average Daily Minimum 2-m Temperature CONUS404-PRISM (C)', cmap='coolwarm', clim=(-6, 6)).opts('Image', alpha=0.6)

CPU times: total: 1.72 s
Wall time: 2.12 s


:DynamicMap   []
   :Overlay
      .WMTS.I  :WMTS   [Longitude,Latitude]
      .Image.I :Image   [lon,lat]   (value)

In [23]:
print(ds_mean.rio.crs)

None


In [24]:
###### Export the raster to GeoTIFF
output_path = "ds_mean_2000-2010.tif"
ds_mean.rio.to_raster(output_path)

import xarray as xr
import rioxarray
from pyproj import CRS

# Load your dataset
ds_mean = xr.open_dataset("ds_mean.tif")

# Print the current CRS (it should be None)
print("Initial CRS:", ds_mean.rio.crs)

# Define the source CRS (WGS 84 for this example)
source_crs = "EPSG:4326"

# Set the CRS using rioxarray
ds_mean.rio.write_crs(source_crs, inplace=True)

# Print the CRS to verify it has been set
print("Updated CRS:", ds_mean.rio.crs)

# # Define the target CRS (e.g., USA Contiguous Lambert Conformal Conic)
# target_crs = CRS.from_proj4(
#     "+proj=lcc +lat_1=33 +lat_2=45 +lat_0=39 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs"
# )

# # Reproject the raster to the target CRS
# ds_mean_reprojected = ds_mean.rio.reproject(target_crs)

# # Export the reprojected raster to GeoTIFF
# output_path = "ds_mean_projected.tif"
# ds_mean_reprojected.rio.to_raster(output_path)

# print("Reprojected CRS:", ds_mean_reprojected.rio.crs)
